In [1]:
import dash
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output, State
import numpy as np
import plotly.io as pio

In [2]:
jobsDF = pd.read_csv('../data/Wuzzuf_Jobs.csv')
jobsDF

,Title,Company,Location,Type,Level,YearsExp,Country,Skills
0,Customer Service Agent,Johnson Controls,New Cairo,Full Time,Entry Level,1-3 Yrs of Exp,Cairo,"Call Center, Customer Support, Customer Care, ..."
1,Marketing Manager,SYE English Community,Dokki,Full Time,Manager,5-10 Yrs of Exp,Giza,"Market Research, Social Media, Online Marketin..."
2,Medical Representative Heliopolis,So Value Pharma,Cairo,Full Time,Entry Level,0-5 Yrs of Exp,Egypt,"Cardio-metabolic, Sales Skills, Sales Target, ..."
3,Order Handling Specialist,Johnson Controls,New Cairo,Full Time,Experienced,3-5 Yrs of Exp,Cairo,"Supply Officer, Supply Planner, Procurement an..."
4,Storekeeper,Confidential,Cairo,Full Time,Experienced,3+ Yrs of Exp,Egypt,"Warehousing, Storekeeping, Logistics/Supply Chain"
...,...,...,...,...,...,...,...,...
4375,Corporate Sales Executive - Hotel Supplies,Term,New Cairo,Full Time,Experienced,3-7 Yrs of Exp,Cairo,"Corporate Sales, Sales Target, English, Comput..."
4376,Logistics Operations & Warehouse Supervisor,DHL Express Egypt,Abu Rawash,Full Time,Manager,6+ Yrs of Exp,Giza,"Warehousing, Supply Chain Management, Customs ..."
4377,English Teacher ( Second Language ) / Lower...,Modern American School of Egypt 2000,New Cairo,Full Time,Experienced,3-7 Yrs of Exp,Cairo,"English Teacher, Lower Primar, Education, Teac..."
4378,Project Manager - Cairo,KnowledgeNet,Nasr City,Full Time,Manager,5+ Yrs of Exp,Cairo,"Implementation, Project Management, Informatio..."


In [3]:
jobsDF = jobsDF.drop_duplicates()

In [4]:
#Dashboard Bans function to calculate the data of columns
def calculate_bans(filter_parameter):
    df = jobsDF.groupby(filter_parameter)[filter_parameter].count().nlargest(1)
    df.to_dict()
    top = df.keys()
    print(top)
    return top

In [5]:
app = Dash(
    external_stylesheets=[dbc.themes.DARKLY], update_title='Loading...'
)

In [6]:
#Dashboard Title
TITLE_STYLE = {
    "position": "flex",
    "top": 0,
    "padding": "2rem 2rem",
}

dashboardTitle = html.Div([
                html.H1("Egypt Job Posts 2022", className="twelve columns", style={"text-align": "center"})
    ], style=TITLE_STYLE,  className = 'row', 
)

In [7]:
#Dashboard Cards
CARD_STYLE={
    "width": "12rem",
    "margin-bottom": "2rem",
}

card1 = dbc.Card(
    [
        
        dbc.CardBody(
            
            html.Div([
            html.P("Top Job Titles", style={"text-align": "center", "text-color": "#F8ECD1"}),
            html.H4(''+calculate_bans('Title'), className="text-danger", style={"text-align": "center"})
            ])
            
        ),
    ],
    style=CARD_STYLE,color="danger", outline=True,
)
card2 = dbc.Card(
    [
        dbc.CardBody(
            
            html.Div([
                html.P("Top Job Companies", style={"text-align": "center", "text-color": "#F8ECD1"}),
                html.H4(''+calculate_bans('Company'), className="text-info", style={"text-align": "center"})
            ])
        ),
        
    ],
    style=CARD_STYLE,color="info", outline=True,
)

card3 = dbc.Card(
    [
        dbc.CardBody(
            
            html.Div([
                html.P("Top Job Locations", style={"text-align": "center", "text-color": "#F8ECD1"}),
                html.H4(''+calculate_bans('Location'), className="text-success", style={"text-align": "center"})
            ])
        ),
        
    ],
    style=CARD_STYLE,color="success", outline=True,
)

card4= dbc.Card(
    [
        dbc.CardBody(
            
            html.Div([
                html.P("Top Job Types", style={"text-align": "center", "text-color": "#F8ECD1"}),
                html.H4(''+calculate_bans('Type'), className="text-primary", style={"text-align": "center"})
            ])
        ),
        
    ],
    style=CARD_STYLE,color="primary", outline=True,
)

card5= dbc.Card(
    [
        dbc.CardBody(
            
            html.Div([
                html.P("Top Experience Years", style={"text-align": "center", "text-color": "#F8ECD1"}),
                html.H4(''+calculate_bans('YearsExp'), className="text-warning", style={"text-align": "center"})
            ])
        ),
        
    ],
    style=CARD_STYLE,color="warning", outline=True,
)

Index(['Accountant'], dtype='object', name='Title')
Index(['Confidential'], dtype='object', name='Company')
Index([' Cairo'], dtype='object', name='Location')
Index(['Full Time'], dtype='object', name='Type')
Index(['1-3 Yrs of Exp'], dtype='object', name='YearsExp')


In [8]:
#filters of the graph
DROPDOWN_STYLE = {
    "margin-bottom": "1rem",
    "margin-top": "1rem",
    "margin-left": "1rem",
    "background-color": "#222",
    "border-color" : "#F8ECD1",
    "color": "#222",
    "fontColor": "#222",
    "text-color": "#222",
    "font-family": "sans-serif",
}

Fiteration = html.Div([
                #html.H3("Filters", className="display-4", style={"text-align": "center"}),
                #dbc.Button("Clear Filters", id="clear_filter", outline=True, color="danger", className="me-1", type='reset', n_clicks=0),
                html.Hr(style={"color":"#F8ECD1"}),
            ])
dropdownYears = html.Div([
                    dcc.Dropdown(
                        id="dropdownGraph1_id",
                        options=[{'label' : str(i), 'value' : str(i)} for i in jobsDF['Location'].unique()],
                        value=None,
                        placeholder='Choose a Location...',
                        multi=True, 
                        style=DROPDOWN_STYLE                        
                    )
                ])
    
dropdownLevel = html.Div([
                    dcc.Dropdown(
                        id="dropdownGraph2_id",
                        options=[{'label' : str(i), 'value' : str(i)} for i in jobsDF['Company'].unique()],
                        value=None,
                        placeholder='Choose a Company....',
                        multi=True, 
                        style=DROPDOWN_STYLE                        
                    )
                ])
dropdownType = html.Div([
                    dcc.Dropdown(
                        id="dropdownGraph3_id",
                        options=[{'label' : str(i), 'value' : str(i)} for i in jobsDF['Type'].unique()],
                        value=None,
                        placeholder='Choose a Job Type....',
                        multi=True, 
                        style=DROPDOWN_STYLE                        
                    )
                ])

dropdownGraph = html.Div([
                    dbc.DropdownMenu(
                        label="Choose Data Display",
                        id="dropdownSelection_id",
                        children=[
                            dbc.DropdownMenuItem("All Data", id="dropdownAll_id"),
                            dbc.DropdownMenuItem("Top 50", id="dropdownTop10_id"),

                        ],style=DROPDOWN_STYLE, color="Info")
])


In [9]:
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [10]:
#Analysis of Jobs Experience Years in Different Companies using Bar plot
GRAPH_STYLE = {
    "width": "70rem",
    "height": "70rem"
}

graph1 = html.Div([
        html.H4('Analysis of Jobs Experience Years in Different Companies'),
        dcc.Graph(id="graph1_id"),
    ], style=GRAPH_STYLE
)

@app.callback(
            Output(component_id='graph1_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value'),
            Input(component_id='dropdownGraph2_id', component_property='value'),
            Input(component_id='dropdownGraph3_id', component_property='value'),
            Input(component_id='dropdownAll_id', component_property='n_clicks'),
            Input(component_id='dropdownTop10_id', component_property='n_clicks')
)
def update_bar_chart(dropdown1_value, dropdown2_value, dropdown3_value, n_clicks1, n_clicks2):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = "all"
    else:
        button_id = ctx.triggered[0]["prop_id"].split(".")[0]
        
    if button_id == 'dropdownAll_id':
        jobs = jobsDF
    else:
        jobs = jobsDF.head(50)
        
    if (dropdown1_value == None) and (dropdown2_value == None) and (dropdown3_value == None):
        fig = px.bar(jobs, x='YearsExp', y='Company', color='Location', opacity=1, template='plotly_dark')

    else:
        if (dropdown1_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Company'].isin(dropdown2_value)]

        elif (dropdown2_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Location'].isin(dropdown1_value)]

        elif (dropdown1_value == None) and (dropdown2_value == None):
            filteredDF = jobs[(jobs['Type'].isin(dropdown3_value))]

        elif (dropdown1_value == None):
            filteredDF = jobs[(jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]

        elif dropdown2_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Type'].isin(dropdown3_value))]

        elif dropdown3_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value))]

        else:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]

        fig = px.bar(filteredDF, x='YearsExp', y='Company', color='Location', opacity=1, template='plotly_dark')
    return fig



In [11]:
#Analysis of Jobs Titles vs. Experience Years in Different Locations using Scatter plot
graph2 = html.Div([
        html.H4('Analysis of Jobs Titles vs. Experience Years in Different Locations'),
        dcc.Graph(id="graph2_id"),
    ], style=GRAPH_STYLE
)

@app.callback(
            Output(component_id='graph2_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value'),
            Input(component_id='dropdownGraph2_id', component_property='value'),
            Input(component_id='dropdownGraph3_id', component_property='value'),
            Input(component_id='dropdownAll_id', component_property='n_clicks'),
            Input(component_id='dropdownTop10_id', component_property='n_clicks')
)
def update_scatter_chart(dropdown1_value, dropdown2_value, dropdown3_value, n_clicks1, n_clicks2):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = "all"
    else:
        button_id = ctx.triggered[0]["prop_id"].split(".")[0]
        
    if button_id == 'dropdownAll_id':
        jobs = jobsDF
    else:
        jobs = jobsDF.head(50)
        
    if (dropdown1_value == None) and (dropdown2_value == None) and (dropdown3_value == None):
        fig = px.scatter(jobs, x='Title', y='YearsExp', width=1100, height=800, color='Location', symbol='Location', opacity=1, template='plotly_dark')
        
    else:
        if (dropdown1_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Company'].isin(dropdown2_value)]

        elif (dropdown2_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Location'].isin(dropdown1_value)]
        
        elif (dropdown1_value == None) and (dropdown2_value == None):
            filteredDF = jobs[(jobs['Type'].isin(dropdown3_value))]
            
        elif (dropdown1_value == None):
            filteredDF = jobs[(jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        elif dropdown2_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Type'].isin(dropdown3_value))]
            
        elif dropdown3_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value))]
            
        else:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        fig = px.scatter(filteredDF, x='Title', y='YearsExp', width=1100, height=800, color='Location', symbol='Location', opacity=1, template='plotly_dark')        
    return fig



In [12]:
#Analysis of Jobs Titles using Histogram
graph3 = html.Div([
        html.H4('Analysis of Jobs Titles in Different Locations'),
        dcc.Graph(id="graph3_id"),
    ], style=GRAPH_STYLE
)

@app.callback(
            Output(component_id='graph3_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value'),
            Input(component_id='dropdownGraph2_id', component_property='value'),
            Input(component_id='dropdownGraph3_id', component_property='value'),
            Input(component_id='dropdownAll_id', component_property='n_clicks'),
            Input(component_id='dropdownTop10_id', component_property='n_clicks')
)
def update_histogram(dropdown1_value, dropdown2_value, dropdown3_value, n_clicks1, n_clicks2):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = "all"
    else:
        button_id = ctx.triggered[0]["prop_id"].split(".")[0]
        
    if button_id == 'dropdownAll_id':
        jobs = jobsDF
    else:
        jobs = jobsDF.head(50)
        
    if (dropdown1_value == None) and (dropdown2_value == None) and (dropdown3_value == None):
        fig = px.bar(jobs, x='Title', width=1100, height=800, color='Location', hover_name='Title', opacity=1, template='plotly_dark')
        
    else:
        if (dropdown1_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Company'].isin(dropdown2_value)]

        elif (dropdown2_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Location'].isin(dropdown1_value)]
        
        elif (dropdown1_value == None) and (dropdown2_value == None):
            filteredDF = jobs[(jobs['Type'].jobs(dropdown3_value))]
            
        elif (dropdown1_value == None):
            filteredDF = jobs[(jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        elif dropdown2_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Type'].isin(dropdown3_value))]
            
        elif dropdown3_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value))]
            
        else:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        fig = px.bar(filteredDF, x='Title', width=1100, height=800, color='Location', hover_name='Title', opacity=1, template='plotly_dark')        
    return fig



In [13]:
#Analysis of Jobs Companies in Different Locations using Histogram
graph4 = html.Div([
        html.H4('Analysis of Jobs Companies in Different Locations'),
        dcc.Graph(id="graph4_id"),
    ], style=GRAPH_STYLE
)

@app.callback(
            Output(component_id='graph4_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value'),
            Input(component_id='dropdownGraph2_id', component_property='value'),
            Input(component_id='dropdownGraph3_id', component_property='value'),
            Input(component_id='dropdownAll_id', component_property='n_clicks'),
            Input(component_id='dropdownTop10_id', component_property='n_clicks')
)
def update_histogram(dropdown1_value, dropdown2_value, dropdown3_value, n_clicks1, n_clicks2):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = "all"
    else:
        button_id = ctx.triggered[0]["prop_id"].split(".")[0]
        
    if button_id == 'dropdownAll_id':
        jobs = jobsDF
    else:
        jobs = jobsDF.head(50)
        
    if (dropdown1_value == None) and (dropdown2_value == None) and (dropdown3_value == None):
        fig = px.bar(jobs, x='Company', width=1100, height=800, color='Location', opacity=1, template='plotly_dark')
        
    else:
        if (dropdown1_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Company'].isin(dropdown2_value)]

        elif (dropdown2_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Location'].isin(dropdown1_value)]
        
        elif (dropdown1_value == None) and (dropdown2_value == None):
            filteredDF = jobs[(jobs['Type'].isin(dropdown3_value))]
            
        elif (dropdown1_value == None):
            filteredDF = jobs[(jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        elif dropdown2_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Type'].isin(dropdown3_value))]
            
        elif dropdown3_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value))]
            
        else:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        fig = px.bar(filteredDF, x='Company', width=1100, height=800, color='Location', opacity=1, template='plotly_dark')        
    return fig



In [14]:
#Analysis of Jobs Companies vs, Titles for Different Types using Scatter plot
graph5= html.Div([
        html.H4('Analysis of Jobs Companies vs. Titles for Different Types'),
        dcc.Graph(id="graph5_id"),
    ], style=GRAPH_STYLE
)

@app.callback(
            Output(component_id='graph5_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value'),
            Input(component_id='dropdownGraph2_id', component_property='value'),
            Input(component_id='dropdownGraph3_id', component_property='value'),
            Input(component_id='dropdownAll_id', component_property='n_clicks'),
            Input(component_id='dropdownTop10_id', component_property='n_clicks')
)
def update_bar_chart(dropdown1_value, dropdown2_value, dropdown3_value, n_clicks1, n_clicks2):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = "all"
    else:
        button_id = ctx.triggered[0]["prop_id"].split(".")[0]
        
    if button_id == 'dropdownAll_id':
        jobs = jobsDF
    else:
        jobs = jobsDF.head(50)
        
    if (dropdown1_value == None) and (dropdown2_value == None) and (dropdown3_value == None):
        fig = px.scatter(jobs, x='Title', y='Company', color='Type', opacity=1, template='plotly_dark')

    else:
        if (dropdown1_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Company'].isin(dropdown2_value)]

        elif (dropdown2_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Location'].isin(dropdown1_value)]

        elif (dropdown1_value == None) and (dropdown2_value == None):
            filteredDF = jobs[(jobs['Type'].isin(dropdown3_value))]

        elif (dropdown1_value == None):
            filteredDF = jobs[(jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]

        elif dropdown2_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Type'].isin(dropdown3_value))]

        elif dropdown3_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value))]

        else:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]

        fig = px.scatter(filteredDF, x='Title', y='Company', color='Type', opacity=1, template='plotly_dark')
    return fig



In [15]:
#Analysis of Jobs Locations in different Copmanies using Bar plot
graph6 = html.Div([
        html.H4('Analysis of Jobs Locations in different Copmanies'),
        dcc.Graph(id="graph6_id"),
    ], style=GRAPH_STYLE
)

@app.callback(
            Output(component_id='graph6_id', component_property='figure'),
            Input(component_id='dropdownGraph1_id', component_property='value'),
            Input(component_id='dropdownGraph2_id', component_property='value'),
            Input(component_id='dropdownGraph3_id', component_property='value'),
            Input(component_id='dropdownAll_id', component_property='n_clicks'),
            Input(component_id='dropdownTop10_id', component_property='n_clicks')
)
def update_scatter_chart(dropdown1_value, dropdown2_value, dropdown3_value, n_clicks1, n_clicks2):
    ctx = dash.callback_context

    if not ctx.triggered:
        button_id = "all"
    else:
        button_id = ctx.triggered[0]["prop_id"].split(".")[0]
        
    if button_id == 'dropdownAll_id':
        jobs = jobsDF
    else:
        jobs = jobsDF.head(50)
        
    if (dropdown1_value == None) and (dropdown2_value == None) and (dropdown3_value == None):
        fig = px.bar(jobs, x='Location', y='Company', color='YearsExp', opacity=1, template='plotly_dark')
        
    else:
        if (dropdown1_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Company'].isin(dropdown2_value)]

        elif (dropdown2_value == None) and (dropdown3_value == None):
            filteredDF = jobs[jobs['Location'].isin(dropdown1_value)]
        
        elif (dropdown1_value == None) and (dropdown2_value == None):
            filteredDF = jobs[(jobs['Type'].isin(dropdown3_value))]
            
        elif (dropdown1_value == None):
            filteredDF = jobs[(jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        elif dropdown2_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Type'].isin(dropdown3_value))]
            
        elif dropdown3_value == None:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value))]
            
        else:
            filteredDF = jobs[(jobs['Location'].isin(dropdown1_value)) & (jobs['Company'].isin(dropdown2_value)) & (jobs['Type'].isin(dropdown3_value))]
        
        fig = px.bar(filteredDF, x='Location', y='Company', color='YearsExp', opacity=1, template='plotly_dark')
    return fig



In [16]:
#Tabs that will the graphs be displayed in
tabsOfGraphs = html.Div([
        dbc.Tabs([
            dbc.Tab(label="Job Titles vs. Experience Years", tab_id="scatter", active_label_style={"color": "#F8ECD1"}),
            dbc.Tab(label="Top Job Experience Years", tab_id="bar", active_label_style={"color": "#F8ECD1"}),
            dbc.Tab(label="Top Job Titles", tab_id="bar1", active_label_style={"color": "#F8ECD1"}),
            dbc.Tab(label="Top Job Types", tab_id="scatter1", active_label_style={"color": "#F8ECD1"}),
            dbc.Tab(label="Top Job Companies", tab_id="bar2", active_label_style={"color": "#F8ECD1"}),
            dbc.Tab(label="Top Job Locations", tab_id="bar3", active_label_style={"color": "#F8ECD1"}),
        ],
        id="tabs",
        active_tab="scatter",
        ),
        html.Div(id="tab-content", className="p-4")
])

@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "active_tab")],
)
def render_tab_content(active_tab):
    """
    This callback takes the 'active_tab' property as input, as well as the
    stored graphs, and renders the tab content depending on what the value of
    'active_tab' is.
    """
    if active_tab is not None:
        if active_tab == "scatter":
            return dbc.Row([
                    dbc.Col(graph2),
                ])
        elif active_tab == "bar":
            return dbc.Row([
                    dbc.Col(graph1),
                ])
        elif active_tab == "bar1":
            return dbc.Row([
                    dbc.Col(graph3),
                ])
        elif active_tab == "bar2":
            return dbc.Row([
                    dbc.Col(graph4),
                ])
        elif active_tab == "scatter1":
            return dbc.Row([
                    dbc.Col(graph5),
                ])
        elif active_tab == "bar3":
            return dbc.Row([
                    dbc.Col(graph6),
                ])
    return "No tab selected"




In [17]:
#Layout of the Dashboard
app.layout = html.Div([
            dashboardTitle, 
    
            dbc.Row([
                dbc.Col(card1, width="auto"),
                dbc.Col(card2, width="auto"),
                dbc.Col(card3, width="auto"),
                dbc.Col(card4, width="auto"),
                dbc.Col(card5, width="auto"),
            ],justify="center"), 
            
            dbc.Row([
                dbc.Col(Fiteration),
                
            ]),
    
            dbc.Row([
                dbc.Col(dropdownYears),
                dbc.Col(dropdownLevel),
                dbc.Col(dropdownType),
                dbc.Col(dropdownGraph),
            ],justify="center"),
    
            dbc.Row([
                dbc.Col(tabsOfGraphs),
            ])                   
        ])

In [ ]:
app.title = 'Egypt Job Posts 2022'
if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2022 22:25:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 22:25:29] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_0m1647776574.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 22:25:29] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_3_0m1647776574.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 22:25:29] "GET /_dash-component-suites/dash/deps/react@16.v2_3_0m1647776574.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 22:25:29] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_3_0m1647776574.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 22:25:29] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_0_3m1648451819.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 22:25:29] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_3_0m1647776574.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 22